In [ ]:
import numpy as np
import lammps
from lammps import LMP_STYLE_ATOM, LMP_TYPE_ARRAY
import ctypes

# This a wrapper function and should be used with caution:
# Periodic datasets with  lammps-format style atomic is currently supported
# Wishlist:- a fitSNAP and LAMMPS-provided descriptor extraction function 

def SNAP(lmp_struc, cutfac, rfrac0, twojmax, R, w):
    '''
    Hyper-parmaters:

    cutfac = scale factor applied to all cutoff radii (positive real) : float

    rfac0 = parameter in distance to angle conversion (0 < rcutfac < 1) : float

    twojmax = band limit for bispectrum components (non-negative integer) : int

    R_1, R_2,… = list of cutoff radii, one for each type (distance units) : list

    w_1, w_2,… = list of neighbor weights, one for each type : list
    
    Returns
    --------
    BiSpec: Bispectrum components for each atoms
    '''

    lmp = lammps.lammps(cmdargs=['-sc','.lammps_output.txt', '-l', 'None'])
    
    R_max = max(R)
    R = ' '.join(map(str, R))
    w = ' '.join(map(str, w))
    
    
    doctring = f"""
    #LAMMPS input file
    # ------------------------ INITIALIZATION ----------------------------
    processors    * * *
    units         metal
    dimension    3
    boundary    p    p    p
    atom_style   atomic
    box tilt large
    #--------------------------- LAMMPS Data File -------------------------
    read_data     {lmp_struc}
    mass * 1.0
    #--------------------------- DUMMY FORCE_FIELD -------------------------
    pair_style      zero {cutfac*R_max*2}
    pair_coeff      * *  
    
    """
    try:
        lmp.commands_string(doctring)
    except Exception as e:
        raise RuntimeError(f'LAMMPS: {e}\n')

    #-------- Compute SNAP descriptors ---------------------        
    lmp.command(f'compute BiSpec all sna/atom {cutfac} {rfrac0} {twojmax} {R} {w}')
    
    try:
        lmp.command('run 0')
    except Exception as e:
        raise RuntimeError(f'LAMMPS: {e}\n')

	#-------- Extract SNAP descriptors ---------------------
    Natoms = lmp.get_natoms()
    print ((f'Number of atoms in the system: {Natoms}'))
    
    if twojmax==8:
        snap_shape = (Natoms, 55)
    elif twojmax==6:
        snap_shape = (Natoms, 30)
    else:
         ValueError(f'Unsupported twojmax value: {twojmax}. Supported values are 6 or 8.')
          
    BiSpec = np.ctypeslib.as_array(lmp.extract_compute('BiSpec', LMP_STYLE_ATOM, LMP_TYPE_ARRAY), shape=snap_shape)
    t = lmp.gather_atoms('type',0,1)
    types = np.ctypeslib.as_array(t)
    
    lmp.close()
    
    return BiSpec, types



def lammps_minimizer(lmp_struc):

        lmp = lammps.lammps(cmdargs=['-sc','.lammps_output.txt'])
        
        doctring = f"""
        #LAMMPS input file
        # ------------------------ INITIALIZATION ----------------------------
        processors    * * *
        units         real
        dimension    3
        boundary    p    p    p
        atom_style   charge
        box tilt large
        #--------------------------- LAMMPS Data File -------------------------
        read_data     {lmp_struc}
        mass * 1.0
        #--------------------------- Reax FORCE_FIELD -------------------------
        pair_style reax/c NULL safezone 16.0 mincap 2400
        pair_coeff * * ffield_start Mo S
        fix 1 all qeq/reax 1 0.0 10.0 1.0e-6 reax/c
        """
        try:
                lmp.commands_string(doctring)
        except Exception as e:
                raise RuntimeError(f'LAMMPS: {e}\n')

        lmp.command('minimize 1e-5 1e-4 10000 10000')
        lmp.command(f'write_data min_{lmp_struc}')
        pe = lmp.get_thermo("pe")
        lmp.close()

        return pe

In [22]:
fname = 'WSe_Structures/2X_1T_uc.lmp'
bispec_descr, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5,5], w=[1,1])

Number of atoms in the system: 216


ValueError: '&<d' is not a valid PEP 3118 buffer format string

In [14]:
fname = 'WSe_Structures/1600000_2X_1T_1500K_scale.lmp'
bispec_descr_1, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5,5], w=[1,1])

TypeError: as_array() requires a shape argument when called on a pointer

In [3]:
fname = 'WSe_Structures/2X_2H_uc.lmp'
bispec_descr_2, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5,5], w=[1,1])

In [ ]:
fname = 'WSe_Structures/1600000_2X_1T_1500K_scale1.0476.lmp'
bispec_descr_3, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5,5], w=[1,1])

In [ ]:
fname = 'WSe_Structures/1600000_2X_2H_1500K_scale1.0476.lmp'
bispec_descr_4, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5,5], w=[1,1])

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(bispec_descr_1.mean(axis=0), label='1T_1500')
plt.semilogy(bispec_descr.mean(axis=0), label='1T_pr')
plt.semilogy(bispec_descr_3.mean(axis=0), label='1T_1500_strain')
plt.semilogy(bispec_descr_2.mean(axis=0), label='2H_pr')
plt.semilogy(bispec_descr_4.mean(axis=0), label='2H_1500_strain')
plt.ylim(1e3, 1e5)
plt.xlim(0, 10)
plt.legend()

In [ ]:
bispec_descr_4

In [ ]:
import numpy as np
import lammps
from lammps import LMP_STYLE_ATOM, LMP_TYPE_ARRAY
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

# --- Cell 0: Define functions ---

def SNAP(lmp_struc, cutfac, rfrac0, twojmax, R, w):
	'''
	Hyperparameters:
	  cutfac: scale factor applied to all cutoff radii (float),
	  rfrac0: parameter in distance-to-angle conversion (float),
	  twojmax: band limit for bispectrum components (int),
	  R: list of cutoff radii (list),
	  w: list of neighbor weights (list)
	  
	Returns
	  BiSpec: Bispectrum components for each atom (numpy.ndarray)
	'''
	lmp = lammps.lammps(cmdargs=['-sc','.lammps_output.txt', '-l', 'None'])
	
	R_max = max(R)
	R = ' '.join(map(str, R))
	w = ' '.join(map(str, w))
	
	doctring = f"""
	#LAMMPS input file
	# ------------------------ INITIALIZATION ----------------------------
	processors    * * *
	units         metal
	dimension    3
	boundary    p    p    p
	atom_style   atomic
	box tilt large
	#--------------------------- LAMMPS Data File -------------------------
	read_data     {lmp_struc}
	mass * 1.0
	#--------------------------- DUMMY FORCE_FIELD -------------------------
	pair_style      zero {cutfac*R_max*2}
	pair_coeff      * *  
	"""
	try:
		lmp.commands_string(doctring)
	except Exception as e:
		raise RuntimeError(f'LAMMPS: {e}\n')

	lmp.command(f'compute BiSpec all sna/atom {cutfac} {rfrac0} {twojmax} {R} {w}')
	
	try:
		lmp.command('run 0')
	except Exception as e:
		raise RuntimeError(f'LAMMPS: {e}\n')

	BiSpec = lmp.extract_compute('BiSpec', LMP_STYLE_ATOM, LMP_TYPE_ARRAY)
	t = lmp.gather_atoms('type', 0, 1)
	types = np.ctypeslib.as_array(t)
	
	lmp.close()
	
	return BiSpec, types


def lammps_minimizer(lmp_struc):
	lmp = lammps.lammps(cmdargs=['-sc','.lammps_output.txt'])
	
	doctring = f"""
		#LAMMPS input file
		# ------------------------ INITIALIZATION ----------------------------
		processors    * * *
		units         real
		dimension    3
		boundary    p    p    p
		atom_style   charge
		box tilt large
		#--------------------------- LAMMPS Data File -------------------------
		read_data     {lmp_struc}
		mass * 1.0
		#--------------------------- Reax FORCE_FIELD -------------------------
		pair_style reax/c NULL safezone 16.0 mincap 2400
		pair_coeff * * ffield_start Mo S
		fix 1 all qeq/reax 1 0.0 10.0 1.0e-6 reax/c
		"""
	try:
		lmp.commands_string(doctring)
	except Exception as e:
		raise RuntimeError(f'LAMMPS: {e}\n')

	lmp.command('minimize 1e-5 1e-4 10000 10000')
	lmp.command(f'write_data min_{lmp_struc}')
	pe = lmp.get_thermo("pe")
	lmp.close()

	return pe

import sys
figure_name = sys.argv[1]
# --- Cell 1: Compute bispectrum descriptors for the first structure ---
fname = 'WSe_Structures/2X_1T_uc.lmp'
bispec_descr, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5, 5], w=[1, 1])

# --- Cell 2: Compute bispectrum descriptors for the second structure ---
fname = 'WSe_Structures/1600000_2X_1T_1500K_scale.lmp'
bispec_descr_1, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5, 5], w=[1, 1])

# --- Cell 3: Compute bispectrum descriptors for the third structure ---
fname = 'WSe_Structures/2X_2H_uc.lmp'
bispec_descr_2, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5, 5], w=[1, 1])

# --- Cell 4: Compute bispectrum descriptors for the fourth structure ---
fname = 'WSe_Structures/1600000_2X_1T_1500K_scale1.0476.lmp'
bispec_descr_3, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5, 5], w=[1, 1])

# --- Cell 5: Compute bispectrum descriptors for the fifth structure ---
fname = 'WSe_Structures/1600000_2X_2H_1500K_scale1.0476.lmp'
bispec_descr_4, _ = SNAP(lmp_struc=fname, cutfac=1, rfrac0=0.5, twojmax=8, R=[5, 5], w=[1, 1])

# --- New Cell at index 7: (currently empty, reserved for additional code) ---


# --- Cell 6 (and/or Cell 8): Plotting the averaged bispectrum descriptor data ---
plt.semilogy(bispec_descr_1.mean(axis=0), label='1T_1500')
plt.semilogy(bispec_descr.mean(axis=0), label='1T_pr')
plt.semilogy(bispec_descr_3.mean(axis=0), label='1T_1500_strain')
plt.semilogy(bispec_descr_2.mean(axis=0), label='2H_pr')
plt.semilogy(bispec_descr_4.mean(axis=0), label='2H_1500_strain')
plt.ylim(1e3, 1e5)
plt.xlim(0, 10)
plt.legend()
plt.savefig(f'{figure_name}.png')





In [8]:
def compute_decriptor_PCA(data, num_PC=3):
	"""
	Reduces descriptor dimensions by first standardizing the data and then applying PCA.
	
	Returns
	-------
	x: Principal components
	variances: Explained variance ratio of the principal components.
	"""
	from sklearn.decomposition import PCA
	from sklearn.preprocessing import StandardScaler
	scaling = StandardScaler()
	scaling.fit(data)
	Scaled_data = scaling.transform(data)
	principal = PCA(n_components=num_PC)
	principal.fit(Scaled_data)
	x = principal.transform(Scaled_data)
	variances = principal.explained_variance_ratio_
	return x, variances



In [9]:
bispec_2_pca = compute_decriptor_PCA(bispec_descr_2, num_PC=3)
bispec_pca = compute_decriptor_PCA(bispec_descr, num_PC=3)

/Users/sny/anaconda3/envs/auto_synth/lib/python3.11/site-packages/sklearn/utils/extmath.py:1120: RuntimeWarning: overflow encountered in square
  temp **= 2
/Users/sny/anaconda3/envs/auto_synth/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/Users/sny/anaconda3/envs/auto_synth/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/Users/sny/anaconda3/envs/auto_synth/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:81: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# plt.plot(bispec_2_pca[0][:,0], label='2H_pr')
plt.plot(bispec_pca[0][:,0], label='1T_pr')
plt.plot(bispec_pca[0][:,1], label='1T_pr')

In [10]:
bispec_descr

array([[0.00000000e+000, 1.52320439e-320, 2.68277646e-321, ...,
                    nan, 7.33948374e-296, 1.07034138e-296],
       [            nan,             nan,             nan, ...,
        4.45085393e-307, 2.70744993e-300,             nan],
       [            nan,             nan, 7.26302497e-297, ...,
        1.43349292e-298,             nan,             nan],
       ...,
       [1.98628350e-315, 0.00000000e+000, 0.00000000e+000, ...,
        1.98632411e-315, 3.17805381e-314, 1.98628363e-315],
       [0.00000000e+000, 0.00000000e+000, 3.17811859e-314, ...,
        3.17805403e-314, 1.98628377e-315, 0.00000000e+000],
       [0.00000000e+000, 3.17811881e-314, 1.98632425e-315, ...,
        1.98628390e-315, 0.00000000e+000, 0.00000000e+000]])

In [11]:
bispec_descr_2

array([[ 9.95161384e+04,  8.27440615e+04,  3.47468970e+04, ...,
        -6.67179101e-02, -1.52574091e+01, -6.14193871e+00],
       [ 9.97210384e+04,  8.33258290e+04,  3.49698510e+04, ...,
        -8.83756134e+00, -8.57578163e+00, -9.50516730e+00],
       [ 9.97212143e+04,  8.33260053e+04,  3.49699332e+04, ...,
        -8.83758756e+00, -8.57555358e+00, -9.50529632e+00],
       ...,
       [ 9.97216539e+04,  8.33265293e+04,  3.49702970e+04, ...,
        -8.83727094e+00, -8.57583097e+00, -9.50524589e+00],
       [ 9.97214993e+04,  8.33263597e+04,  3.49701903e+04, ...,
        -8.83753633e+00, -8.57568286e+00, -9.50530443e+00],
       [ 9.95158260e+04,  8.27436944e+04,  3.47466556e+04, ...,
        -6.71418887e-02, -1.52571749e+01, -6.14205761e+00]])

In [6]:
bispec_descr_1

NameError: name 'bispec_descr_1' is not defined